# Example code for QPARC Challenge 2022

In [2]:
# Import some modules.
# You are not allowed to import qulacs.QuantumState, nor other quantum circuit simulators.

from typing import Counter

import numpy as np
from openfermion.transforms import jordan_wigner
from qulacs import QuantumCircuit
from qulacs.gate import CZ, RY, H, Sdag, CNOT
from scipy.optimize import minimize
import matplotlib.pyplot as plt

from qparc import (
    QulacsExecutor,
    create_observable_from_openfermion_text,
    TotalShotsExceeded,
)

import mylibs
import importlib
from qulacs import QuantumState
from openfermion.linalg import get_sparse_operator
from openfermion.ops import QubitOperator

In [3]:
# Set up the executor, and get the problem hamiltonian.
# One must run quantum circuits always through the executor.
executor = QulacsExecutor()
fermionic_hamiltonian, n_qubits = executor.get_problem_hamiltonian()


# Process the Hamiltonian.
jw_hamiltonian = jordan_wigner(fermionic_hamiltonian)
qulacs_hamiltonian = create_observable_from_openfermion_text(str(jw_hamiltonian))

In [4]:
# One can obtain HF energy and FCI energy.
print("HF energy:", executor.hf_energy)
print("FCI energy:", executor.fci_energy)

HF energy: -2.0985459369977626
FCI energy: -2.1663874486347625


In [5]:
# Set up the ansatz


def ry_ansatz_circuit(n_qubits, depth, theta_list):
    """ry_ansatz_circuit
    Returns Ry ansatz circuit.

    Args:
        n_qubits:
            The number of qubit used
        depth:
            Depth of the circuit.
        theta_list:
            Rotation angles.
    Returns:
        circuit:
            Resulting Ry ansatz circuit.
    """
    circuit = QuantumCircuit(n_qubits)
    params_id = 0
    for d in range(depth):
        for i in range(n_qubits // 2):
            circuit.add_gate(RY(2 * i, theta_list[params_id]))
            params_id += 1
        for i in range(n_qubits // 4):
            circuit.add_gate(CZ(4 * i, 4 * i + 2))
        for i in range(n_qubits // 4):
            circuit.add_gate(CZ(4 * i + 2, (4 * i + 4) % n_qubits))
    for i in range(n_qubits // 2):
        circuit.add_gate(RY(2 * i, theta_list[params_id]))
        params_id += 1
    for i in range(n_qubits // 2):
        circuit.add_gate(CNOT(2 * i, 2 * i + 1))

    return circuit

In [6]:
def get_terms_and_measurement_circuits(observable):
    """get_terms_and_measurement_circuits
    Returns basis-rotation circuits for measurement, along with the corresponding terms.

    Args:
        observable:
            The observable to be measured.
    Returns:
        pauli_coef:
            List of coefficients.
        pauli_target:
            List of targetted qubits.
        pauli_gate:
            List of circuits for basis-rotation.
    """
    pauli_coef = []
    pauli_target = []
    pauli_gate = []
    n_qubits = observable.get_qubit_count()
    for i_term in range(observable.get_term_count()):
        term = observable.get_term(i_term)
        pauli_coef.append(term.get_coef())
        target_list = term.get_index_list()
        pauli_target.append(target_list)
        id_list = term.get_pauli_id_list()
        circuit = QuantumCircuit(n_qubits)
        for target, id in zip(target_list, id_list):
            if id == 1:
                circuit.add_gate(H(target))
            elif id == 2:
                circuit.add_gate(Sdag(target))
                circuit.add_gate(H(target))
            elif id == 3:
                pass
            else:
                raise Exception(f"Operator {target, id} not supported")
        pauli_gate.append(circuit)
    return pauli_coef, pauli_target, pauli_gate

In [9]:
def get_energy(theta_list, n_shots, depth, state, hamiltonian):
    """get_energy
    Returns the evaluated energy

    Args:
        theta_list:
            The parameters
        n_shots:
            The number of shots used to evaluate each term.
        depth:
            The depth of the ansatz
        state:
            The integer that defines the initial state in the computational basis.
        hamiltonian:
            The Hamiltonian to be evaluated.
    Returns:
        ret:
            The evaluated energy.
    """
    pauli_coef, pauli_target, pauli_gate = get_terms_and_measurement_circuits(
        hamiltonian
    )
    n_qubits = hamiltonian.get_qubit_count()
    circuit = ry_ansatz_circuit(n_qubits=n_qubits, depth=depth, theta_list=theta_list)
    ret = 0
    for coef, target, gate in zip(pauli_coef, pauli_target, pauli_gate):
        if target:
            counts = Counter(
                executor.sampling(
                    [circuit, gate],
                    state_int=state,
                    n_qubits=n_qubits,
                    n_shots=n_shots,
                )
            )
            for sample, count in counts.items():
                binary = np.binary_repr(sample).rjust(n_qubits, "0")
                measurement = np.product(
                    [-1 if binary[n_qubits - t - 1] == "1" else 1 for t in target]
                )
                ret += coef * measurement * count / n_shots
        else:
            ret += coef

    return ret.real

In [109]:
# not used in the example
def get_gradient(theta_list, n_shots, depth, state, hamiltonian):
    """get_gradient
    Returns the evaluated gradient of the energy in the parameter space.

    Args:
        theta_list:
            The parameters
        n_shots:
            The number of shots used to evaluate each term.
        depth:
            The depth of the ansatz
        state:
            The integer that defines the initial state in the computational basis.
        hamiltonian:
            The Hamiltonian to be evaluated.
    Returns:
        np.array(g):
            The gradient of the energy in the parameter space.
    """
    g = []

    param_dim = len(theta_list)
    for i in range(param_dim):
        shift = np.zeros(param_dim)
        shift[i] = 0.5 * np.pi
        gi = 0.5 * (
            get_energy(
                theta_list=theta_list + shift,
                n_shots=n_shots,
                depth=depth,
                state=state,
                hamiltonian=hamiltonian,
            )
            - get_energy(
                theta_list=theta_list - shift,
                n_shots=n_shots,
                depth=depth,
                state=state,
                hamiltonian=hamiltonian,
            )
        )
        g.append(gi)
    return np.array(g)

In [106]:
def compute_coefficient(theta_list, n_shots, depth, state, hamiltonian):
    n_param = len(theta_list)
    E = dict()
    E["a"] = get_energy(theta_list, n_shots["a"], depth, state, hamiltonian)
 
    E["b"] = np.zeros(n_param)
    E["c"] = np.zeros(n_param)
    std_basis_set = np.identity(n_param)
    for i in range(n_param):
        E["b"][i] = get_energy(theta_list + np.pi/2*std_basis_set[i], n_shots["b"], depth, state, hamiltonian) \
                  - get_energy(theta_list - np.pi/2*std_basis_set[i], n_shots["b"], depth, state, hamiltonian)
        E["c"][i] = get_energy(theta_list + np.pi*std_basis_set[i], n_shots["c"], depth, state, hamiltonian)

    E["d"] = np.zeros((n_param,n_param))
    for k in range(n_param):
        for l in range(k+1,n_param):
            E["d"][k][l] = get_energy(theta_list + np.pi/2*std_basis_set[k] + np.pi/2*std_basis_set[l], n_shots["d"], depth, state, hamiltonian) \
                         + get_energy(theta_list - np.pi/2*std_basis_set[k] - np.pi/2*std_basis_set[l], n_shots["d"], depth, state, hamiltonian) \
                         - get_energy(theta_list - np.pi/2*std_basis_set[k] + np.pi/2*std_basis_set[l], n_shots["d"], depth, state, hamiltonian) \
                         - get_energy(theta_list + np.pi/2*std_basis_set[k] - np.pi/2*std_basis_set[l], n_shots["d"], depth, state, hamiltonian)
 
    return E

In [107]:
# Define functions to be used in the optimization process.
def cost(theta_list, state=initial_state):
    ret = get_energy(
        theta_list=theta_list,
        n_shots=n_shots,
        depth=depth,
        state=state,
        hamiltonian=qulacs_hamiltonian,
    )
    # executor.current_value will be used as the final result when the number of shots reach the limit,
    # so set the current value as often as possible, if you find a better energy.
    if ret < executor.current_value:
        executor.current_value = ret
    return ret


def grad(theta_list):
    ret = get_gradient(
        theta_list=theta_list,
        n_shots=n_shots,
        depth=depth,
        state=initial_state,
        hamiltonian=qulacs_hamiltonian,
    )
    return ret


def callback(theta_list):
    pass

In [43]:
# Define input settings
#n_shots = 10000
n_shots = {"a":1000, "b":10000, "c":1000, "d":0}
#initial_state = 0b00001111
initial_state = 0b00000000
depth = 2

In [46]:
# Define functions to be used in the optimization process.
def cost(theta_list, state=initial_state):
    ret = get_energy(
        theta_list=theta_list,
        n_shots=n_shots,
        depth=depth,
        state=state,
        hamiltonian=qulacs_hamiltonian,
    )
    # executor.current_value will be used as the final result when the number of shots reach the limit,
    # so set the current value as often as possible, if you find a better energy.
    if ret < executor.current_value:
        executor.current_value = ret
    # print("current val", executor.current_value)
    # print(theta_list)
    return ret
    

In [15]:
depth = 2
initial_state = 0b00000101

In [112]:
executor.reset()
n_shots = {"a":10000, "b":10000//1, "c":10000//2, "d":1000//2}
init_theta_list = np.zeros(n_qubits//2 * (depth + 1))
E = compute_coefficient(init_theta_list, n_shots, depth, initial_state, qulacs_hamiltonian)
opt = mylibs.approx_optimize(E, np.zeros(len(init_theta_list)), 1)
ans = get_energy(init_theta_list+opt.x, 100000, depth, initial_state, qulacs_hamiltonian)
print(ans)

-2.1112567835885745
[-0.0358956  -0.39335819 -0.25849619  0.01106534  0.08074208 -0.17334049
 -0.28145248 -0.13890608  0.11330168  0.19962399 -0.03924928 -0.14026322]


In [117]:
executor.reset()

for i in range(10):
    n_shots = {"a":5000, "b":4000, "c":1000, "d":550}
    init_theta_list = np.zeros(n_qubits//2 * (depth + 1))
    E = compute_coefficient(init_theta_list, n_shots, depth, initial_state, qulacs_hamiltonian)
    opt = mylibs.approx_optimize(E, np.zeros(len(init_theta_list)), 1)

    init_theta_list = init_theta_list + opt.x

    E = compute_coefficient(init_theta_list, n_shots, depth, initial_state, qulacs_hamiltonian)
    opt = mylibs.approx_optimize(E, np.zeros(len(init_theta_list)), 1)

    ans = get_energy(init_theta_list+opt.x, 25000, depth, initial_state, qulacs_hamiltonian)
    print(ans)
    executor.current_value = ans
    executor.record_result(verbose=False)
 
executor.evaluate_final_result()

-2.1235674726719047
-2.1141338961441036
-2.1086298747748144
-2.1132810106254944
-2.1182551147091533
-2.1037502338032987
-2.1178734282861003
-2.1128498249492336
-2.1211289123944925
-2.1175245899990927

############## Final Result ##############
Average energy: -2.115099435835769
Average accuracy: 0.051288012798993685
------------------------------------------
FCI energy = -2.1663874486347625
HF energy = -2.0985459369977626
##########################################
